**Librerias**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pylab as plt
import math
import plotly.express as px

**Carga de Datos**

In [ ]:

df = pd.read_csv("C:/Users/eserrano/Desktop/Maestria/Analitica_Predictiva/dataset2.csv")
df.head()

**Analisis Exploratorio**

In [ ]:
print('Cantidad de Filas y columnas:',df.shape)
print('Nombre columnas:',df.columns)

In [ ]:
##Columnas, nulos y tipo de datos
df.info()

In [ ]:
## Valores Duplicados
df.duplicated().sum()

In [ ]:
## Cantidad nulos 
df.isnull().sum()

In [ ]:
## Valores unicos de la etiqueta activityID
df['activityID'].unique(), df['activityID'].nunique()

In [ ]:
## descripción estadística de los datos numéricos
df.describe().T

In [ ]:
## Matriz de Correlación
corr_mat = df.corr()
plt.figure(figsize=(25, 20))
sns.heatmap(corr_mat, annot=True)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
## Tiempo por actividad

features = ['heart_rate','hand temperature (°C)','chest acceleration Y ±16g', 'chest acceleration Z ±16g',]
df1 = df[df.PeopleId == 1]
t = np.array([0.01*i for i in range(len(df1))])
class_labels = np.unique(df1["activityID"])
colors = ['red', 'green', 'blue', 'orange', 'purple', 'cyan', 'magenta', 'yellow', 'pink', 'brown', 'gray', 'lime', 'teal']

for feature in features:

    feat = df1[feature]



    plt.figure(figsize=(15, 9))
    for i, class_label in enumerate(class_labels):
        mask = (df1["activityID"] == class_label)
        plt.scatter(t[mask], feat[mask], s=2.5, c=colors[i], label=class_label)
    plt.legend(title='Activity',fontsize='large',markerscale=5)

    plt.xlabel('Time (s)')
    plt.ylabel(feature)
    plt.title(f'Scatter Plot of {feature}')

    plt.show()

# **Análisis Predictivo**

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
X = df.iloc[:, 1:-1].values
y = df.iloc[:, 0].values
print(X)
print(y)

In [ ]:
le = LabelEncoder()
y_temp = le.fit_transform(y)
print(list(le.classes_))
counter = Counter(y_temp)

print(counter)
for k,v in counter.items():
    per = v / len(y_temp) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

In [ ]:
label_encoder = LabelEncoder()
df_copy = df.copy()
df_copy['activityID'] = label_encoder.fit_transform(df_copy[['activityID']])

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X)
X = imputer.transform(X)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)


In [ ]:
## Matriz de Confusión
fig = cm
plt.figure(figsize=(25, 20))
sns.heatmap(fig, annot=True, cmap='Blues')
plt.title('Confusión Heatmap')
plt.show()

In [ ]:
imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
df_copy['heart_rate'] = imputer.fit_transform(df_copy[['heart_rate']])
df_copy.isnull().sum()

In [ ]:
y = df_copy['activityID']
x =  df_copy.drop('activityID', axis = 1)

In [ ]:
y.head()

In [ ]:
x.head()

In [ ]:
x.info()

In [ ]:
x.describe().T

In [ ]:
X_continuous = x.drop('PeopleId', axis = 1)
X_cat = x['PeopleId']
X_continuous.head()

In [ ]:
f, ax = plt.subplots(math.ceil(len(X_continuous.columns)/2),2,figsize = (20,60))

ax = ax.ravel()

for i, con_col in enumerate(X_continuous.columns):
    #ax[i].figure(figsize = (8,10))
    ax[i].hist(X_continuous[con_col], bins = 15)
    ax[i].set_title(f'{con_col}')

plt.tight_layout()
plt.show()

In [ ]:
X_continuous.skew()